In [24]:
import pandas as pd
from pandas.tseries.offsets import BDay
import statsmodels.api as sm

### 1. Merge COT Data with Futures Return

#### WTI Crude Oil

In [25]:
# --- Load and clean the COT data ---
df_cot = pd.read_csv("WTI.csv")
df_cot["COT_Date"] = pd.to_datetime(df_cot["Report_Date_as_YYYY_MM_DD"])

# --- Load and clean front-month futures prices (CL1) ---
df_price_raw = pd.read_excel("CL1.xlsx", skiprows=6)
df_price_clean = df_price_raw[["Date", "PX_SETTLE"]].dropna()
df_price_clean.columns = ["Date", "Settle"]
df_price_clean["Date"] = pd.to_datetime(df_price_clean["Date"])

# --- Build expiry calendar for WTI contracts ---
# Note: CME WTI contracts expire 3 business days before the 25th of the month prior to delivery.
def get_expiry_dates(start, end):
    dates = pd.date_range(start, end, freq="MS")
    expiry = [pd.Timestamp(y, m, 25) - BDay(3) for y, m in zip(dates.year, dates.month)]
    expiry_df = pd.DataFrame({"Expiry": expiry})
    expiry_df["YearMonth"] = expiry_df["Expiry"].dt.to_period("M")
    return expiry_df
expiry_calendar = get_expiry_dates(df_price_clean["Date"].min(), df_price_clean["Date"].max())

# --- Assign each price to a contract month and flag rollover weeks ---
df_price_clean["YearMonth"] = df_price_clean["Date"].dt.to_period("M")
df_price_clean = df_price_clean.merge(expiry_calendar, on="YearMonth", how="left")
# Compute days-to-expiry
df_price_clean["DaysToExpiry"] = (df_price_clean["Expiry"] - df_price_clean["Date"]).dt.days
df_price_clean["RolloverRisk"] = df_price_clean["DaysToExpiry"].between(0, 5) # Any price date within 5 calendar days of expiry is flagged

# --- Merge COT data with prices and drop rollover weeks ---
df_merged = pd.merge(df_cot, df_price_clean, left_on="COT_Date", right_on="Date", how="inner")
df_merged_clean = df_merged[~df_merged["RolloverRisk"]].copy()

# --- Calculate weekly returns ---
df_merged_clean = df_merged_clean.sort_values("COT_Date")
df_merged_clean["Settle_t+1"] = df_merged_clean["Settle"].shift(-1)
df_merged_clean["Return"] = (df_merged_clean["Settle_t+1"] - df_merged_clean["Settle"]) / df_merged_clean["Settle"]



/var/folders/x9/81t79ql10xlfpj2hrqlppjj80000gn/T/ipykernel_75682/113116674.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_cot["COT_Date"] = pd.to_datetime(df_cot["Report_Date_as_YYYY_MM_DD"])


In [26]:
useful_columns = [
    "COT_Date", "Settle", "Settle_t+1", "Return","Open_Interest_All",
    # Producer/Merchant
    "Prod_Merc_Positions_Long_All", "Prod_Merc_Positions_Short_All", "Prod_Net",
    # Swap Dealer
    "Swap_Positions_Long_All", "Swap__Positions_Short_All", "Swap_Net",
    # Managed Money
    "M_Money_Positions_Long_All", "M_Money_Positions_Short_All", "MM_Net",
    # Other Reportables
    "Other_Rept_Positions_Long_All", "Other_Rept_Positions_Short_All","Other_Net",
    # Non-Reportables
    "NonRept_Positions_Long_All", "NonRept_Positions_Short_All","NonRep_Net"
]
df_merged_clean["MM_Net"] = df_merged_clean["M_Money_Positions_Long_All"] - df_merged_clean["M_Money_Positions_Short_All"]
df_merged_clean["Swap_Net"] = df_merged_clean["Swap_Positions_Long_All"] - df_merged_clean["Swap__Positions_Short_All"]
df_merged_clean["Prod_Net"] = df_merged_clean["Prod_Merc_Positions_Long_All"] - df_merged_clean["Prod_Merc_Positions_Short_All"]
df_merged_clean["Other_Net"] = df_merged_clean["Other_Rept_Positions_Long_All"] - df_merged_clean["Other_Rept_Positions_Short_All"]
df_merged_clean["NonRep_Net"] = df_merged_clean["NonRept_Positions_Long_All"] - df_merged_clean["NonRept_Positions_Short_All"]

df_WTI_final = df_merged_clean[useful_columns].copy()

#### RBOB Gasoline

In [27]:
# --- Load and prepare RBOB position data ---
df_gas = pd.read_csv("Gasoline.csv")
df_gas["COT_Date"] = pd.to_datetime(df_gas["Report_Date_as_YYYY_MM_DD"])

# --- Load and prepare XB1 front-month futures price data ---
df_xb1_raw = pd.read_excel("XB1.xlsx", skiprows=6)
df_xb1_clean = df_xb1_raw[["Date", "PX_SETTLE"]].dropna()
df_xb1_clean.columns = ["Date", "Settle"]
df_xb1_clean["Date"] = pd.to_datetime(df_xb1_clean["Date"])

# --- Build expiry calendar for RBOB Gasoline ---
# RBOB contracts expire on the last business day of the month before the contract month
def get_rbob_expiry_dates(start, end):
    dates = pd.date_range(start, end, freq="MS")
    expiry = [pd.Timestamp(y, m, 1) - BDay(1) for y, m in zip(dates.year, dates.month)]
    expiry_df = pd.DataFrame({"Expiry": expiry})
    expiry_df["YearMonth"] = expiry_df["Expiry"].dt.to_period("M")
    return expiry_df

rbob_expiry_calendar = get_rbob_expiry_dates(df_xb1_clean["Date"].min(), df_xb1_clean["Date"].max())

# --- Flag rollover risk ---
df_xb1_clean["YearMonth"] = df_xb1_clean["Date"].dt.to_period("M")
df_xb1_clean = df_xb1_clean.merge(rbob_expiry_calendar, on="YearMonth", how="left")
df_xb1_clean["DaysToExpiry"] = (df_xb1_clean["Expiry"] - df_xb1_clean["Date"]).dt.days
df_xb1_clean["RolloverRisk"] = df_xb1_clean["DaysToExpiry"].between(0, 5)

# --- Merge COT data with XB1 price data and exclude rollover weeks ---
df_merged = pd.merge(df_gas, df_xb1_clean, left_on="COT_Date", right_on="Date", how="inner")
df_merged_clean = df_merged[~df_merged["RolloverRisk"]].copy()

# --- Calculate weekly returns ---
df_merged_clean = df_merged_clean.sort_values("COT_Date")
df_merged_clean["Settle_t+1"] = df_merged_clean["Settle"].shift(-1)
df_merged_clean["Return"] = (df_merged_clean["Settle_t+1"] - df_merged_clean["Settle"]) / df_merged_clean["Settle"]

# --- Calculate net positions for all five trader types ---
df_merged_clean["MM_Net"] = df_merged_clean["M_Money_Positions_Long_All"] - df_merged_clean["M_Money_Positions_Short_All"]
df_merged_clean["Swap_Net"] = df_merged_clean["Swap_Positions_Long_All"] - df_merged_clean["Swap__Positions_Short_All"]
df_merged_clean["Prod_Net"] = df_merged_clean["Prod_Merc_Positions_Long_All"] - df_merged_clean["Prod_Merc_Positions_Short_All"]
df_merged_clean["Other_Net"] = df_merged_clean["Other_Rept_Positions_Long_All"] - df_merged_clean["Other_Rept_Positions_Short_All"]
df_merged_clean["NonRep_Net"] = df_merged_clean["NonRept_Positions_Long_All"] - df_merged_clean["NonRept_Positions_Short_All"]

useful_columns = [
    "COT_Date", "Settle", "Settle_t+1", "Return","Open_Interest_All",
    "Prod_Merc_Positions_Long_All", "Prod_Merc_Positions_Short_All", "Prod_Net",
    "Swap_Positions_Long_All", "Swap__Positions_Short_All", "Swap_Net",
    "M_Money_Positions_Long_All", "M_Money_Positions_Short_All", "MM_Net",
    "Other_Rept_Positions_Long_All", "Other_Rept_Positions_Short_All", "Other_Net",
    "NonRept_Positions_Long_All", "NonRept_Positions_Short_All", "NonRep_Net"
]

df_XB_final = df_merged_clean[useful_columns].copy()
df_XB_final.to_excel("XB_merged.xlsx", index=False)

/var/folders/x9/81t79ql10xlfpj2hrqlppjj80000gn/T/ipykernel_75682/3021542581.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_gas["COT_Date"] = pd.to_datetime(df_gas["Report_Date_as_YYYY_MM_DD"])


### 2. Feature transforms (net positions as % change and z-scores)

In [28]:
# --- Convert to % change ---
net_cols = ["MM_Net", "Swap_Net", "Prod_Net", "Other_Net", "NonRep_Net"]

# --- For RBOB ---
for col in net_cols:
    df_XB_final[f"{col}_Chg"] = df_XB_final[col].pct_change()
    df_XB_final[f"{col}_Chg"] =df_XB_final[f"{col}_Chg"].replace([float("inf"), float("-inf")], pd.NA)

# --- For WTI ---
for col in net_cols:
    df_WTI_final[f"{col}_Chg"] = df_WTI_final[col].pct_change()
    df_WTI_final[f"{col}_Chg"] = df_WTI_final[f"{col}_Chg"].replace([float("inf"), float("-inf")], pd.NA)

df_WTI_final.to_excel("WTI_merged.xlsx", index=False)
df_XB_final.to_excel("XB_merged.xlsx", index=False)

In [34]:
# --- Convert to z-scores ---

# Rolling Z‑scores (52‑week window)
window = 52
def add_z(df):
    for col in net_cols:
        df[col+"_z"] = (df[col] - df[col].rolling(window, min_periods=26).mean()) / df[col].rolling(window, min_periods=26).std()
    return df

df_WTI = add_z(df_WTI_final)
df_XB  = add_z(df_XB_final)

z_cols = [c+"_z" for c in net_cols]

### 3. Regression Model

In [35]:
def get_R2_and_models(df, label):
    R2s = {}
    models = {}
    for h in [1,2,3]:
        tgt = f"Ret_t{h}"
        df[tgt] = df["Return"].shift(-h)
        reg = df[[tgt]+z_cols].dropna()
        X = sm.add_constant(reg[z_cols])
        res = sm.OLS(reg[tgt], X).fit()
        R2s[h] = res.rsquared
        models[h] = res
    return R2s, models

R2_wti, models_wti = get_R2_and_models(df_WTI,"WTI")
R2_xb , models_xb  = get_R2_and_models(df_XB ,"RBOB")

R2_table_WTI = pd.DataFrame({
    "Lag": ["t+1","t+2","t+3"],
    "R2": [round(R2_wti[1],4), round(R2_wti[2],4), round(R2_wti[3],4)]
})
R2_table_XB = pd.DataFrame({
    "Lag": ["t+1","t+2","t+3"],
    "R2": [round(R2_xb[1],4), round(R2_xb[2],4), round(R2_xb[3],4)]
})

# Select best horizon 
best_wti_h = max(R2_wti, key=R2_wti.get)
best_xb_h  = max(R2_xb , key=R2_xb.get)

def beta_table(res, label, horizon):
    rows = []
    for var in z_cols:
        rows.append({
            "Predictor": var.replace("_Net_z","").replace("_z",""),
            "Beta": round(res.params[var],4),
            "p_value": round(res.pvalues[var],4)
        })
    return pd.DataFrame(rows)

beta_wti = beta_table(models_wti[best_wti_h], "WTI", best_wti_h)
beta_xb  = beta_table(models_xb[best_xb_h], "RBOB", best_xb_h)

In [36]:
R2_table_WTI

,Lag,R2
0,t+1,0.0072
1,t+2,0.0093
2,t+3,0.0058


In [37]:
R2_table_XB

,Lag,R2
0,t+1,0.0125
1,t+2,0.0140
2,t+3,0.0106


In [38]:
beta_wti

,Predictor,Beta,p_value
0,MM,0.0043,0.1457
1,Swap,0.0054,0.1822
2,Prod,0.0042,0.3336
3,Other,0.0006,0.8747
4,NonRep,0.0025,0.3043


In [39]:
beta_xb

,Predictor,Beta,p_value
0,MM,-0.0054,0.3298
1,Swap,0.0004,0.8901
2,Prod,-0.0108,0.0478
3,Other,-0.0013,0.7151
4,NonRep,-0.0033,0.3003
